In [2]:
#CODE TO GET BUREAU LEVEL COMPARISONS (THIS WAS BEFORE I FULLY UNDERSTOOD PROJECT)
import pandas as pd
import os

In [76]:
#FEB 2024 CBO SPENDING PROJECTIONS
currentyr = 2024
years = [i for i in range(currentyr, currentyr+11)] 
cbo_ot = pd.read_excel('data/cbo-2024-02-Spending-Projections.xlsx', header=7, sheet_name=1)
cbo_ot = cbo_ot[[col for col in cbo_ot.columns if col not in years]] #remove the BA cols
cbo_ot.columns = cbo_ot.columns.str.replace('.1', '')
cbo_ot = cbo_ot.drop(columns=['Unnamed: 9', "Unnamed: 21", "Off-Budget?"]).drop(index=0).drop(cbo_ot.index[-2:])
cbo_ot['25-34'] = cbo_ot.loc[:, '2025':'2034'].sum(axis=1)
#cbo_ot["Agency-Bureau-Program"] = cbo_ot["Agency"] + "BUREAU: " + cbo_ot["Bureau"] + "PROGRAM: " + cbo_ot["Title"]
cbo_ot["Agency-Bureau"] = cbo_ot["Agency"] + "BUREAU: " + cbo_ot["Bureau"]
agency_bureau_10yr_2024 = cbo_ot[["Agency-Bureau", "25-34"]].groupby("Agency-Bureau").sum()
agency_bureau_10yr_2024

,25-34
Agency-Bureau,
400 Years of African-American History Commission BUREAU: 400 Years of African-American History Commission,32.0
Access Board BUREAU: Access Board,119.0
Administrative Conference of the United States BUREAU: Administrative Conference of the United States,36.0
Advisory Council on Historic Preservation BUREAU: Advisory Council on Historic Preservation,108.0
Affordable Housing Program BUREAU: Affordable Housing Program,2506.0
...,...
United Mine Workers of America Benefit Funds BUREAU: United Mine Workers of America Benefit Funds,5072.0
United States Court of Appeals for Veterans Claims BUREAU: United States Court of Appeals for Veterans Claims,589.0
United States Holocaust Memorial Museum BUREAU: United States Holocaust Memorial Museum,734.0


In [46]:
#FEB 2021 CBO SPENDING PROJECTIONS
cbo21_ot = pd.read_excel('data/cbo-2021-02-11-spendingprojections.xlsx', header=7, sheet_name=1)
years_21 = [i for i in range(2021, 2032)]
cbo21_ot = cbo21_ot[[col for col in cbo21_ot.columns if col not in years_21]].drop(columns=["Off-Budget?", "Unnamed: 9", "Unnamed: 21"]) #remove the BA cols, unnamed and unnecessary columns
cbo21_ot.columns = cbo21_ot.columns.str.replace('.1', '') #clean OT columns
cbo21_ot["22-31"] = cbo21_ot.loc[:, '2022':'2031'].sum(axis=1) #sum the years
cbo21_ot["Agency-Bureau"] = cbo21_ot["Agency"] + "BUREAU: " + cbo21_ot["Bureau"] #create a new column for agency-bureau
agency_bureau_10yr_2021 = cbo21_ot[["Agency-Bureau", "22-31"]].groupby("Agency-Bureau").sum() #group by agency-bureau and sum the total

In [75]:
#merge the two dataframes
merged = pd.merge(agency_bureau_10yr_2021, agency_bureau_10yr_2024, on="Agency-Bureau", how="inner")
merged['percent_change'] = (merged['25-34'] - merged['22-31']) / merged['22-31'] * 100
merged.reset_index(inplace=True)

merged["Agency"] = merged["Agency-Bureau"].str.extract(r'(.*)BUREAU:')
merged["Bureau"] = merged["Agency-Bureau"].str.extract(r'BUREAU:\s(.*)')
merged["Agency"] = merged["Agency"].str.strip()
merged["Bureau"] = merged["Bureau"].str.strip()

merged = merged[['Agency-Bureau', 'Agency', 'Bureau', '22-31', '25-34', 'percent_change']]
merged = merged.drop(columns=["Agency-Bureau"])

#merged.to_excel("output/cbo_spending_changes_S1.xlsx", index=False)
merged.shape

(343, 5)

In [ ]:
with pd.ExcelWriter("output/cbo_spending_changes_S1.xlsx", mode='a', engine="openpyxl") as writer:
    merged.to_excel(writer, index=False)